# 

In [1]:
%env EVALUATION_SYSTEM_CONFIG_DIR=/work/ch1187/clint/nextgems/freva/
%env EVALUATION_SYSTEM_CONFIG_FILE=/work/ch1187/clint/nextgems/freva/evaluation_system.conf

env: EVALUATION_SYSTEM_CONFIG_DIR=/work/ch1187/clint/nextgems/freva/
env: EVALUATION_SYSTEM_CONFIG_FILE=/work/ch1187/clint/nextgems/freva/evaluation_system.conf


In [2]:
from freva_client import databrowser

In [3]:
databrowser.metadata_search(fs_type="s3")

{'ensemble': ['r1i1p1f1'],
 'experiment': ['historical', 'ssp370'],
 'institute': ['ncar'],
 'model': ['cesm2-le'],
 'product': ['output'],
 'project': ['cesm2-le'],
 'realm': ['atm', 'ice', 'lnd', 'ocn'],
 'time_aggregation': ['mean'],
 'time_frequency': ['daily', 'monthly', 'static'],
 'variable': ['aice',
  'aice_d',
  'dic',
  'doc',
  'flns',
  'flnsc',
  'flut',
  'fsno',
  'fsns',
  'fsnsc',
  'fsntoa',
  'h2osno',
  'hi',
  'hi_d',
  'icefrac',
  'lhflx',
  'npp',
  'o2',
  'pd',
  'precc',
  'precl',
  'precsc',
  'precsl',
  'ps',
  'psl',
  'q',
  'qrunoff',
  'rain',
  'salt',
  'shflx',
  'snow',
  'soilliq',
  'soilwater_10cm',
  't',
  'temp',
  'tmq',
  'trefht',
  'trefhtmn',
  'trefhtmx',
  'trefmxav',
  'ts',
  'u',
  'ues',
  'uet',
  'uvel',
  'v',
  'vns',
  'vnt',
  'vvel',
  'wts',
  'wtt',
  'wvel',
  'z3']}

In [4]:
databrowser.metadata_search(fs_type="s3")['project']

['cesm2-le']

In [5]:
for facet, values in databrowser.metadata_search(fs_type='s3').items():
    if facet in ('realm','time_frequency','experiment'):
        print(facet,'\n\t',values)

experiment 
	 ['historical', 'ssp370']
realm 
	 ['atm', 'ice', 'lnd', 'ocn']
time_frequency 
	 ['daily', 'monthly', 'static']


In [6]:
search_keys = {
    'fs_type': 's3',
    'project':'cesm2-le',
    'experiment': 'historical',
    'realm': 'atm',
    'time_frequency':'monthly'
}
'ts' in databrowser.metadata_search(**search_keys)['variable']

True

In [7]:
db = databrowser(variable='ts', **search_keys)
db,list(db)[:5]

(databrowser(flavour=freva, host=http://nextgems.dkrz.de:7777/api/freva-nextgen/databrowser, multi_version=False, variable=ts, fs_type=s3, project=cesm2-le, experiment=historical, realm=atm, time_frequency=monthly),
 ['s3://ncar-cesm2-lens/atm/monthly/cesm2LE-historical-smbb-TS.zarr',
  's3://ncar-cesm2-lens/atm/monthly/cesm2LE-historical-cmip6-TS.zarr'])

In [8]:
import xarray as xr
import numpy as np
def field_mean(
    data: xr.DataArray,
    lat_name: str = "lat",
    lon_name: str = "lon",
    mean_dims: tuple[str, str] = ("lat", "lon")
) -> xr.DataArray:
    """
    Compute an area-weighted mean over latitude and longitude using cosine latitude weights.

    Parameters
    ----------
    data : xr.DataArray
        Input data array with latitude and longitude coordinates.
    lat_name : str, optional
        Name of the latitude coordinate. Default is "lat".
    lon_name : str, optional
        Name of the longitude coordinate. Default is "lon".
    mean_dims : Sequence[str], optional
        Dimensions over which to compute the mean. Default is ("lat", "lon").

    Returns
    -------
    xr.DataArray
        Area-weighted mean of the input data over the specified dimensions.
    """
    # Extract latitude values
    lat = data[lat_name]

    # Compute cosine of latitude in radians
    weights = np.cos(np.deg2rad(lat))

    # Normalize weights so they sum to 1 (over just lat)
    weights /= weights.sum()

    # Apply weighted mean over specified dimensions
    return data.weighted(weights).mean(dim=mean_dims, keep_attrs=True)

In [9]:
xr.open_zarr(list(db)[0], storage_options={ 'anon':True })

<xarray.Dataset> Size: 22GB
Dimensions:    (member_id: 50, time: 1980, lat: 192, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 2kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * member_id  (member_id) <U12 2kB 'r10i1191p1f2' ... 'r9i1171p1f2'
  * time       (time) object 16kB 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bnds  (time, nbnd) object 32kB dask.array<chunksize=(1980, 2), meta=np.ndarray>
Dimensions without coordinates: nbnd
Data variables:
    TS         (member_id, time, lat, lon) float32 22GB dask.array<chunksize=(1, 600, 192, 288), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    logname:           sunseon
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    source:            CAM
    time_period_freq:  month_1
    topography_file:   /mnt/lustre/share/CESM/cesm_input/atm/cam/topo/fv_0.9x...

In [10]:
s3_opts = { 'anon':True }
from pathlib import Path
time_series = {}

for fileurl in db:
    print(f"Opening {fileurl}...")
    engine = {'engine':'zarr'} if fileurl.endswith('zarr') else {}
    %time ds = xr.open_dataset(fileurl, **engine, storage_options=s3_opts)
    ds = ds.rename({'member_id':'ensemble'})
    ds
    
    # Going through all ensembles might take some time
    # let's make a cut at 5 member for demo purposes
    # Memory usage might spike to ~70GB 
    members=ds.ensemble[:5]
    ds = ds.sel(ensemble=members)    
    
    dataset_name=Path(fileurl).stem   

    
    mean_ts = field_mean(ds["TS"])
    ds = xr.Dataset()
    ds.attrs['source_dataset'] = fileurl
    time_series[dataset_name] = mean_ds
   

Opening s3://ncar-cesm2-lens/atm/monthly/cesm2LE-historical-smbb-TS.zarr...
CPU times: user 28.1 ms, sys: 2.88 ms, total: 31 ms
Wall time: 1.82 s


NameError: name 'mean_ds' is not defined

In [ ]:
import matplotlib.pyplot as plt

# Create a color cycle for different experiments
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

plt.figure(figsize=(12, 5))

for i, (exp, data) in enumerate(time_series.items()):
    ts = data.resample(time="1YE").mean()
    ts_min = ts.min(dim="ensemble").squeeze()
    ts_max = ts.max(dim="ensemble").squeeze()
    ts_mean = ts.mean(dim="ensemble").squeeze()
    # Plot min–max shading
    plt.fill_between(
        ts.time.values,
        ts_min,
        ts_max,
        color=colors[i % len(colors)],
        alpha=0.2,
        label=None,
    )
    # Plot mean line
    plt.plot(
        ts.time,
        ts_mean,
        color=colors[i % len(colors)],
        linewidth=2,
        label=f"{exp}"
    )

# Add plot decorations
plt.title("Ensemble Mean and Spread for Each Dataset")
plt.xlabel("Time")
plt.ylabel(f"{data.attrs['long_name']} [{data.attrs['units']}]")  # Replace with actual units
plt.legend()
plt.grid(True)
plt.tight_layout()

# We need to save the figure before showing
import io
fig_buf = io.BytesIO()
fig_format = 'png'
plt.savefig(fig_buf, format=fig_format)

# Finally plot 
plt.show()

# Let's save the plot on S3, as well as the datasets!

In [ ]:
from os import environ # to get USER env variable
USERNAME=environ['USER']
s3_config = {
    'bucket' : 'freva',
    'endpoint' :'https://s3.eu-dkrz-1.dkrz.cloud', # DKRZ Minio S3
    'prefix' : f'workshop/{environ['USER']}', # Avoid users writting object with same prefix
    'access_key_id' : "s3handson", # Only valid during the workshop
    'secret_access_key' : "s3handson", # Only valid during the workshop
    'region' : 'eu-dkrz-1',
}

import s3fs
s3 = s3fs.S3FileSystem(
    key=s3_config['access_key_id'],
    secret= s3_config['secret_access_key'],
    client_kwargs={'endpoint_url': s3_config['endpoint']},
)

full_prefix = s3_config['bucket']+'/'+s3_config['prefix'] 
testobj=f'{full_prefix}/hi.txt'
s3.write_bytes(testobj, b'Hi!\n')

print(f"Writing data to S3 works! Test it with:\ncurl {s3_config['endpoint']}/{testobj}")

In [ ]:
!curl https://s3.eu-dkrz-1.dkrz.cloud/freva/workshop/${USER}/hi.txt

## Save the figure on S3
 - Save the figure into a buffer instead of a file
 - Make a `PUT` request to write the _in-memory_ image into an S3 object

In [ ]:
figure_path = f'{full_prefix}/figure-ts-mean.{fig_format}'
fig_buf.seek(0) ## rewind the buffer to the beginning
s3.write_bytes(figure_path, fig_buf.getvalue())
print(f"{s3_config['endpoint']}/{figure_path}")

### Lets write the data to S3 as both NetCDF and ZARR

In [ ]:
def write_dataset_to_s3(name, dataset:xr.Dataset, file_format='nc'):
    _supported_types = ('nc','zarr')
    file_format = file_format.replace('.','')
    if file_format not in _supported_types:
        raise Exception('Unsuported file format, use one of')
    
    ## Copy to s3
    s3_path = f'{full_prefix}/{name}.{file_format}'
    if file_format == 'nc':
        tmp_name=f"/scratch/{USERNAME[0]}/{USERNAME}/{name}.nc"

        dataset.to_netcdf(tmp_name, engine='h5netcdf')
            
        ## Copy to s3
        with s3.open(s3_path,'wb') as s3file:
            with open(tmp_name, 'rb') as tmpf:
                s3file.write(tmpf.read())

        ## Check if we can open with Xarray and CDO!
        with s3.open(s3_path,'rb') as s3file:
            xr.open_dataset(s3file, engine='h5netcdf')

        print(f"Try running:\n\t" \
              f"/fastdata/k20200/k202186/public/bin/ncdump -h {s3_config['endpoint']}/{s3_path}#mode=s3,bytes\n\t"\
              f"/fastdata/k20200/k202186/public/bin/cdo sinfo {s3_config['endpoint']}/{s3_path}#mode=s3,bytes")
        
        ## We can now remove the local copy
        os.remove(tmp_name)

    elif file_format == 'zarr':
        # We need to create a S3 store zarr store
        import s3fs
        asyncs3 = s3fs.S3FileSystem(
            key=s3_config['access_key_id'],
            secret= s3_config['secret_access_key'],
            client_kwargs={'endpoint_url': s3_config['endpoint']},
            asynchronous=True,
        )
        store = zarr.storage.FsspecStore(fs=asyncs3,path='/'+s3_path)
        
        write_zarr(store, dataset)
        print(f"Try running:\n\t" \
              f"/fastdata/k20200/k202186/public/bin/ncdump -h {s3_config['endpoint']}/{s3_path}#mode=s3,zarr\n\t"\
              f"/fastdata/k20200/k202186/public/bin/cdo sinfo {s3_config['endpoint']}/{s3_path}#mode=s3,zarr")
        

In [ ]:
def write_zarr(store, data: xr.Dataset):
    if type(data) == xr.DataArray:
        data= data.to_dataset()
    import zarr
    zarr.config.set(default_zarr_format=2)
    chunk_encoding = { "chunk_key_encoding": zarr.core.chunk_key_encodings.V2ChunkKeyEncoding(separator="/").to_dict() }
    
    def get_encoding(data: xr.Dataset):
        import numcodecs
        codec = numcodecs.Blosc(shuffle=1, clevel=6)
        return {
            var: {
                #"chunks": get_chunks(dataset[var].dims),
                "compressors": codec,
                "chunk_key_encoding": zarr.core.chunk_key_encodings.V2ChunkKeyEncoding(separator="/").to_dict(),
            }
            for var in data.variables
            #if var not in dataset.dims
        }
    data.to_zarr(store, encoding=get_encoding(data),mode='w') # overwrites!!!
    zarr.consolidate_metadata(store)

In [ ]:
for name, data in time_series.items():
    ## for CDO it is important that time is the first dimension
    ## also it cannot have indexers of type string
    ensemble = data.ensemble.values
    ensemble_id = np.arange(len(ensemble))
    
    da = data.to_dataset() \
            .transpose('time',...) \
            .assign_coords(ensemble_id=('ensemble',ensemble_id)) \
            .swap_dims({'ensemble':'ensemble_id'}) \
            .reset_coords(drop=True)\
            .assign_attrs(ensembles= ', '.join(data.ensemble.values))
                
    write_dataset_to_s3(name, da, file_format='zarr')
    #write_zarr('/scratch/k/k202186/test.zarr',da)
da